앙상블 학습

1. 정형 데이터의 예측 분석 영역에서는 매우 높은 예측 성능. Voting, Bagging 과 Boosting
2. 배깅 방식의 대표인 Random Forest는 뛰어난 예측 성능, 상대적으로 빠른 수행시간, 유연성 등으로 애용.
3. 부스팅의 효시는 Gradient Boosting, 한 단계 발전시키면서도 시간 단축시킨 XgBoost, LightGBM이 정형 데이터의 분류 영역에서 활용도 확대
4. 앙상블의 앙상블이라고 불리는 스태킹 기법
5. 앙상블의 기본 알고리즘은 결정 트리

Voting Classifier

In [2]:
import pandas as pd
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
from sklearn.datasets import load_breast_cancer

In [4]:
cancer = load_breast_cancer()

In [5]:
lr_clf = LogisticRegression()
knn_clf = KNeighborsClassifier(n_neighbors=8)

# 개별 모델을 소프트 보팅 기반의 앙상블 모델로 구현한 분류기
vo_clf = VotingClassifier(estimators=[('LR', lr_clf), ('KNN', knn_clf)], \
                          voting='soft')

X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, \
                                                    test_size=0.2, random_state=156)

In [6]:
import warnings

warnings.filterwarnings('ignore')

vo_clf.fit(X_train, y_train)
pred = vo_clf.predict(X_test)
print('Voting 정확도:{0: .4f}'.format(accuracy_score(y_test, pred)))

classifiers = [lr_clf, knn_clf]
for classifier in classifiers:
    classifier.fit(X_train, y_train)
    pred = classifier.predict(X_test)
    class_name = classifier.__class__.__name__
    print('{0} 정확도:{1:.4f}'. format(class_name, \
                                    accuracy_score(y_test, pred)))

Voting 정확도: 0.9474
LogisticRegression 정확도:0.9386
KNeighborsClassifier 정확도:0.9386


Bagging 방식의 대표적인 모델: Randomforest

In [28]:
def get_human_dataset():
    #각 데이터 파일들은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당
    feature_name_df = pd.read_csv('dataset/human_activity/features.txt', sep='\s+', header=None, names=['column_index', 'column_name'])
    
    # DataFrame에 피처명을 칼럼으로 부여하기 위해 리스크 객체로 다시 반환
    feature_name = feature_name_df.iloc[:,1].values.tolist()
    
    #학습 피처 데이터 세트와 테스트 피처 데이터를 DataFrame으로 로딩, 컬럼명은 feature_name 적용
    #우선 칼럼명 없이 데이터프레임으로 불러온 후 칼럼명을 추가함
    X_train = pd.read_csv('dataset/human_activity/train/X_train.txt', sep='\s+', header=None)    
    X_train.columns = feature_name
    X_test = pd.read_csv('dataset/human_activity/test/X_test.txt', sep='\s+', header=None)
    X_test.columns = feature_name
    
    # 학습 레이블과 테스트 레이블 데이터를 DataFrame으로 로딩하고 칼럼명은 action으로 부여
    y_train = pd.read_csv('dataset/human_activity/train/y_train.txt', sep='\s+', header=None, names=['action'])
    y_test = pd.read_csv('dataset/human_activity/test/y_test.txt', sep='\s+', header=None, names=['action'])
    
    # 로드된 학습/테스트용 DataFrame을 모두 반환
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = get_human_dataset()

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import warnings

warnings.filterwarnings('ignore')

rf_clf = RandomForestClassifier(random_state=0)
rf_clf.fit(X_train, y_train)
pred = rf_clf.predict(X_test)
accuracy = accuracy_score(y_test, pred)
print('rf 정확도:{0:.4f}'.format(accuracy))

rf 정확도:0.9253


In [30]:
# RandomForestClassifier parameter 목록 확인
rf_clf.get_params

<bound method BaseEstimator.get_params of RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)>

In [15]:
from sklearn.model_selection import GridSearchCV

params = {'n_estimators': [100], 'max_depth':[6,8,10,12], 'min_samples_leaf':[8], 'min_samples_split':[8]}

#n_jobs: -1: 모든 코어를 사용하겠다
rf_clf = RandomForestClassifier(random_state=0, n_jobs=-1)
grid_cv = GridSearchCV(rf_clf, param_grid=params, cv=2, n_jobs=-1)
grid_cv.fit(X_train, y_train)

print('최적 하이퍼 파라미터: \n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}\n'.format(grid_cv.best_score_))

최적 하이퍼 파라미터: 
 {'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 100}
최고 예측 정확도: 0.9451



In [16]:
best_gr_clf = grid_cv.best_estimator_
pred1 = best_gr_clf.predict(X_test)
accuracy = accuracy_score(y_test, pred1)
print('정확도: {0: .4f}'.format(accuracy))

정확도:  0.9386


In [36]:
import matplotlib.pyplot as plt
import seaborn as sns

ftr_importances_values = best_gr_clf.feature_importances_
ftr_importances = pd.Series(ftr_importances_values, index=X_train.columns)
ftr_top20 = ftr_importances.sort_values(ascending=False)[:20]
plt.figure(figsize=(10,8))
plt.title('Feature importances Top 20')

sns.barplot(x=ftr_top20,y=ftr_top20.index)

plt.show()
plt.close()

ValueError: Length of passed values is 30, index implies 561.

Gradient Boosting Machine

In [18]:
#모델 돌리는 시간 체크
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import time
import warnings

start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train, y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)
print('GBM 정확도: {0:.1f}'.format(gb_accuracy))
print('GBM 수행시간: {0:.1f}'.format(time.time() - start_time))

GBM 정확도: 1.0
GBM 수행시간: 0.4


In [20]:
from sklearn.model_selection import GridSearchCV

# learning_rate는 0.1이 default, 작을수록 학습률이 촘촘해진다.
params = {'n_estimators':[100, 300, 500], 'learning_rate':[0.05, 0.1]}
grid_cv = GridSearchCV(gb_clf, param_grid=params, cv=2, verbose=1)
grid_cv.fit(X_train, y_train)

print('최적 하이퍼파라미터:', grid_cv.best_params_)
print('최고 정확도:{0: .4f}', format(grid_cv.best_score_))

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    3.6s finished


최적 하이퍼파라미터: {'learning_rate': 0.1, 'n_estimators': 100}
최고 정확도:{0: .4f} 0.938499884071412


In [22]:
gb_pred = grid_cv.best_estimator_.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)
print('GBM 정확도:{0:.4f}'.format(gb_accuracy))

GBM 정확도:0.9561
